In [1]:
import os
from tkinter import *
from tkinter import filedialog
import tkinter as tk
import PIL.Image
from PIL import Image, ImageTk
from PIL import ImageDraw # для отрисовки 68 landmarks

In [2]:
import numpy as np
import dlib
import cv2
from matplotlib import pyplot as plt
import shutil # для копирование файлов
import imgaug # для аугментации
from imgaug import augmenters as iaa # для аугментации

# Сериализация

In [3]:
import pickle

class Serialize_DB:
    
    name_of_db = 'db_persons.pkl'
    
    # сохранение данных в файле
    # known_face_encodings - кодировки лиц (128d векторы)
    # known_face_names - именя людей в соответсвии к кодировкам
    # data_labels - номера людей для классификации
    def save_data_to_file(self, known_face_encodings, known_face_names, data_labels):
    
        db_for_serialize = { "encodings" : known_face_encodings,
                             "names" : known_face_names,
                             "labels" : data_labels}
        print("Writing DB to file...")
        pickle.dump(db_for_serialize, open(self.name_of_db, 'wb'))

    # получение данных из файла
    def read_data_from_file(self):
        print("Reading DB from file...")
        db_from_serialize = pickle.load(open(self.name_of_db, 'rb'))
        return db_from_serialize

# Нормализация фотографии

In [4]:
from collections import OrderedDict

FACIAL_LANDMARKS_5_POINTS = OrderedDict([
    ("right_eye", (2, 3)),
    ("left_eye", (0, 1)),
    ("nose", (4))
])

def rect_to_box(rect):
    # преобразование квадрата, предсказанного dlib, 
    # в формат (x, y, w, h), который обычно используется в OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right()
    h = rect.bottom()

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

# dlib object -> np.array
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

class FaceNormalizer:
    
    def __init__(self, predictor, desiredLeftEye=(0.33, 0.33),
        desiredFaceWidth=256, desiredFaceHeight=None):
        # сохранение определителя ключевых точек лица, желаемая позиция левого глаза
        # и желаемая ширина и высота лица на выходящем изображении
        self.predictor_landmarks = predictor
        self.desiredLeftEye = desiredLeftEye
        self.desiredFaceWidth = desiredFaceWidth
        self.desiredFaceHeight = desiredFaceHeight
        # if the desired face height is None, set it to be the
        # desired face width (normal behavior)
        if self.desiredFaceHeight is None:
            self.desiredFaceHeight = self.desiredFaceWidth
            
    # функция для нормализации лица
    # image - изображение в RGB 
    # gray - серое изображение
    # rect - область лица, выделенная HOG
    def align(self, image, gray, rect):
        # получение 5 ориентиров-landmarks
        shape = self.predictor_landmarks(image, rect)
        # преобразование dlib object в np.array
        shape = shape_to_np(shape)
        
        # извлечение левого и правого глаз (x, y)-coordinates
        (lStart, lEnd) = FACIAL_LANDMARKS_5_POINTS["left_eye"]
        (rStart, rEnd) = FACIAL_LANDMARKS_5_POINTS["right_eye"]
        # извлечение точек левого глаза и правого (x, y)-coordinates
        leftEyePts = shape[lStart:lEnd + 1]
        rightEyePts = shape[rStart:rEnd + 1]
        
        # расчет центра для каждого глаза
        leftEyeCenter = leftEyePts.mean(axis=0).astype("int")
        rightEyeCenter = rightEyePts.mean(axis=0).astype("int")
        # расчет угла между центроидами глаз
        dY = rightEyeCenter[1] - leftEyeCenter[1]
        dX = rightEyeCenter[0] - leftEyeCenter[0]
        angle = np.degrees(np.arctan2(dY, dX)) - 180
        #print(angle)
        
        # вычисление х-коорд правого глаза основанной на x-коорд левого глаза
        desiredRightEyeX = 1.0 - self.desiredLeftEye[0]
        # определение масштаба результирующего изображения, взяв
        # отношение расстояния между глазами в текущем изображении
        # к отношению расстояния глаз в желаемом изображении
        dist = np.sqrt((dX ** 2) + (dY ** 2)) # Евклидово расстояние
        desiredDist = (desiredRightEyeX - self.desiredLeftEye[0]) # 
        desiredDist *= self.desiredFaceWidth
        scale = desiredDist / dist
        
        # вычисление центра между глазами (x, y)-coordinates
        # для вращения фотографии вокруг этого центра
        eyesCenter = ((leftEyeCenter[0] + rightEyeCenter[0]) / 2,
        (leftEyeCenter[1] + rightEyeCenter[1]) / 2)
        #print("eyesCenter = " + str(eyesCenter))
        
        # получение матрицы для поворота и масштабирования лица
        M = cv2.getRotationMatrix2D(center=eyesCenter, angle=angle, scale=scale)
        
        # обновление компонентов матрицы на смещение
        tX = self.desiredFaceWidth * 0.5
        tY = self.desiredFaceHeight * self.desiredLeftEye[1]
        M[0, 2] += (tX - eyesCenter[0])
        M[1, 2] += (tY - eyesCenter[1])
        
        # применение аффинного преобразования
        (w, h) = (self.desiredFaceWidth, self.desiredFaceHeight)
        output = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)
        
        # возвращение нормализованного лица
        return output



In [4]:
face_normalizer = FaceNormalizer(landmarks_predictor, desiredFaceWidth=150, desiredFaceHeight=150)

check_img = load_image("klava_koka_ii.jpg")
gray_image = cv2.cvtColor(check_img, cv2.COLOR_BGR2GRAY)
detected_faces = detector(check_img, 1)

normalizedFace = face_normalizer.align(check_img, gray_image, detected_faces[0])

# plt.imshow(normalizedFace)

# face_chip = dlib.get_face_chip(check_img, landmarks_predictor(check_img, detected_faces[0]))
# plt.imshow(face_chip)

NameError: name 'landmarks_predictor' is not defined

# Создание БД кодировок фотографий

In [5]:
# функция для загрузки фотографии
def load_image(path_to_image):
    img = PIL.Image.open(path_to_image)
    
    return np.array(img)

# определение моделей
face_encoder = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")
detector = dlib.get_frontal_face_detector()  # определение области лица с помощью HOG
landmarks_predictor = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')

In [6]:
#функция для добавления вектора лица в БД
def add_to_db(vector_128d, id_num, person_name):
    
    known_face_encodings.append(vector_128d)
    known_face_names.append(person_name)
    data_labels.append(id_num)

# Функция для определения лица и возвращение вектора 128d
def detect_face_and_encode(path_to_img, img_name, image_identify=False):
    
    # ======== lib Face Recognition ============
#     img_to_recognize = face_recognition.load_image_file(path_to_img + "/" + img_name)
#     face_locations = face_recognition.face_locations(img_to_recognize, model="hog")
#     face_encodings = face_recognition.face_encodings(img_to_recognize, face_locations)
    
    if image_identify == False:
        img_to_recognize = load_image(path_to_img + "/" + img_name)
    else:
        img_to_recognize = load_image(path_to_img)
        
    detected_faces = detector(img_to_recognize, 1) # определение области лица HOG
    
    if len(detected_faces) == 0:
        print("Error: There is no faces in image!")
        return []
    
    # определение экземпляра нормализатора лица
    face_normalizer = FaceNormalizer(landmarks_predictor, desiredFaceWidth=150, desiredFaceHeight=150)
    
    # получение нормализованного лица
    gray_image = cv2.cvtColor(img_to_recognize, cv2.COLOR_BGR2GRAY)
    normalizedFace = face_normalizer.align(img_to_recognize, gray_image, detected_faces[0])
    # получение 128d вектора embedding
    face_vector = np.array(face_encoder.compute_face_descriptor(normalizedFace, num_jitters=1))
    return face_vector
    

def create_db():

    # папка с эталонными фото
    root_dir = "images_db"
    count_persons = len(os.listdir(root_dir)) # кол-во человек в БД
    
    # удаление скрытого файла
    if os.path.exists(root_dir + "/.DS_Store"):
        os.remove(root_dir + "/.DS_Store")
    
    id_label = 0
    for person_dir in os.listdir(root_dir):
        print("Process person: " + person_dir + " - " + str(id_label + 1) + "/" + str(count_persons))

        path_to_person = root_dir + "/" + person_dir
        
        # удаление скрытого файла
        if os.path.exists(path_to_person + "/.DS_Store"):
            os.remove(path_to_person + "/.DS_Store")
        
        imgs_in_person_dir = os.listdir(path_to_person)
        
        # если в папке человека 1 фото, то нужна аугментация
        if len(imgs_in_person_dir) == 1:
            print("Process of augmentation...")
            augmentate_image(path_to_person + "/" + imgs_in_person_dir[0])
            
        # обновить список фотографий после аугментации
        imgs_in_person_dir = os.listdir(path_to_person)
        
        # проход по всем фото в папке одного человека
        for image in imgs_in_person_dir:
            print(image)
            
            # определение области лица и получение закодированного лица
            encoded_face = detect_face_and_encode(path_to_person, image)
            # добавление закодированного лица в БД
            if len(encoded_face) == 128:
                add_to_db(encoded_face, id_label, person_dir)

        id_label += 1 # изменения номера отметки для следующего человека

In [7]:
known_face_encodings = [] # кодировки лиц из 128d векторов
known_face_names = [] # соответствующие имена векторам
data_labels = [] # соответствующие метки

saver_DB = Serialize_DB()
# если нужна сохраненная БД
need_new_DB = False

if need_new_DB == False:
    # используется сохраненная база
    data_from_existing_DB = saver_DB.read_data_from_file()
    known_face_encodings = data_from_existing_DB["encodings"]
    known_face_names = data_from_existing_DB["names"]
    data_labels = data_from_existing_DB["labels"]
else:
    # Вызов функции для создания БД: флаг False - если аугментация не нужна
    create_db()
    # сериализация данных в файл
    saver_DB.save_data_to_file(known_face_encodings, known_face_names, data_labels)

Reading DB from file...


# Аугментация фотографии

In [8]:
def augmentate_image(path_to_image):
    
    # получение пути к фото и названия фото
    head_tail = os.path.split(path_to_image)
    path_dir = head_tail[0]
    img_name = head_tail[1]
    
    # загрузка изображения
    img = load_image(path_to_image)
    
    flag_flip = 1.0
    brightness_dark = -10
    brightness_light = 10
    
    for i in range(6):
        aug_pipeline = iaa.Sequential([
        #iaa.Affine(rotate=(-5, 5)),
        #iaa.AdditiveGaussianNoise(scale=(10, 60)),
        iaa.Affine(scale=(0.8, 1.1)),
        iaa.Crop(percent=(0, 0.15)),
        iaa.Fliplr(flag_flip),
        iaa.AddToBrightness((brightness_dark, brightness_light))
        ],
        random_order=True)

        augmented_photo = np.array(aug_pipeline.augment_image(img))
        
        # сохранения аугментированного изображения
        new_img_name = path_dir + "/" + str(i + 1) + ".jpg"
        cv2.imwrite(new_img_name, cv2.cvtColor(augmented_photo, cv2.COLOR_RGB2BGR))
        
        # изменение параметров
        flag_flip = 0.0 if flag_flip == 1.0 else 1.0
        brightness_dark -= 20
    #     brightness_light += 10

In [9]:
#augmentate_image("images_db/Aaron_Piersol/Aaron_Piersol.jpg")

# Функция для выбора фото

In [9]:
def show_image():
    global path_img_to_identify
    
    fln = filedialog.askopenfilename(initialdir=os.getcwd(), title="Вибір фото",
                                    filetypes=(("JPG File", "*.jpg"),
                                               ("JPEG File", "*.jpeg"),
                                               ("PNG File", "*.png"),
                                               ("All files", "*.*")))
    
    img = Image.open(fln)
    img.thumbnail((400,400))
    img = ImageTk.PhotoImage(img)
    photo_left.configure(image=img)
    photo_left.image = img
    path_img_to_identify = fln

# KNN классификатор

In [10]:
# KNN алгоримт
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [11]:
def my_dist(x, y):
    
    diff = np.linalg.norm(x - y)
    if diff > 0.6:
#         print("ZERO")
        return 1
    else:
#         print("GOOD")
        return diff


def train_classifier_KNN():
    global X_train, X_test, y_train, y_test
    global classifier
    
    X_train, X_test, y_train, y_test = train_test_split(known_face_encodings, data_labels, test_size=0.3)
    
    classifier = KNeighborsClassifier(n_neighbors = 3, metric=my_dist)
    classifier.fit(X_train, y_train)
    
    print("Re-FIT KNN!")

In [12]:
train_classifier_KNN()

yhat_class = classifier.predict(X_test)
yhat_class

Re-FIT KNN!


array([4, 3, 9, 5, 2, 8, 8, 8, 2, 2, 4, 5, 7, 3, 0, 9, 1, 8, 6, 1, 2])

In [13]:
y_test

[4, 3, 9, 5, 2, 8, 8, 8, 2, 2, 4, 5, 7, 3, 0, 9, 1, 8, 6, 1, 2]

# Вспомогательные функции

In [14]:
def get_unique_values(values):
    uniques = []
    for v in values:
        if v not in uniques:
            uniques.append(v)
    return uniques

# Функционал для Добавления человека в БД

In [21]:
# открытие проводника для выбора фото
def choose_person_img():
    global path_to_new_person
    
    fln = filedialog.askopenfilename(initialdir=os.getcwd(), title="Вибір фото",
                                    filetypes=(("JPG File", "*.jpg"),
                                               ("JPEG File", "*.jpeg"),
                                               ("PNG File", "*.png"),
                                               ("All files", "*.*")))
    path_to_new_person = fln
    
    
    img_name = os.path.split(path_to_new_person)[1]
    name_choosed_img.config(text="Зображення: " + img_name)
    
    
def process_person_img(path_to_dir, name_person):
    # получение последнего номера класса
    id_new_person = data_labels[-1] + 1
    
    # проход по всем фотографиям нового человека
    for img_in_person_dir in os.listdir(path_to_dir):
        
        # получение закодированного лица
        encoded_face = detect_face_and_encode(path_to_dir, img_in_person_dir)
        # добавление закодированного лица в БД
        if len(encoded_face) == 128:
            add_to_db(encoded_face, id_new_person, name_person)
    
    # сохранение обновленной БД в файл
    saver_DB.save_data_to_file(known_face_encodings, known_face_names, data_labels)
    # переобучение классификатора
    train_classifier_KNN()
            
    
def add_person_to_db():
    global name_choosed_img
    
    new_window = tk.Toplevel(root)
    
    new_window.title("Додавання людини до БД")
    new_window.geometry("480x240")
    new_window.resizable(width=False, height=False)
    new_window.configure(background='#a3c2c2')
    
    name_choosed_img = Label(new_window, text="Зображення не обране!", font="Arial 20", bg='#a3c2c2', fg='#ffffff', borderwidth=2)
    name_choosed_img.place(relx=0.05, rely=0.2)
    
    button_choose_photo = tk.Button(new_window, text = "Обрати фото", command=choose_person_img)
    button_choose_photo.place(relx=0.75, rely=0.2, relwidth=0.2, relheight=0.15)
    
    text_input_name = Label(new_window, text="Введіть им'я: ", font="Arial 20", bg='#a3c2c2', fg='#ffffff', borderwidth=2)
    text_input_name.place(relx=0.10, rely=0.5)
    
    entry_field = Entry(new_window)
    entry_field.place(relx=0.40, rely=0.5, relwidth=0.5)
    
    def add_func():
        new_dir_name = "images_db/" + entry_field.get()
        # создание новой папки для человека
        os.mkdir(new_dir_name)
        # копирование в папку выбранной фотографии
        shutil.copyfile(path_to_new_person, new_dir_name + "/" + entry_field.get() + ".jpg")
        
        new_path_to_image = new_dir_name + "/" + entry_field.get() + ".jpg"
        # выполнение аугментации
        augmentate_image(new_path_to_image)
        
        # получить 128-признаки из аугментированных фото
        process_person_img(new_dir_name, entry_field.get())
        
        new_window.destroy()
    
    button_add = Button(new_window, text = "Додати", command=add_func)
    button_add.place(relx=0.4, rely=0.8, relwidth=0.2, relheight=0.15)
    
    

# Функция для кнопки Распознать

In [17]:
def recognize_person():
    
    flag_found = False
    
#     img_to_recognize = face_recognition.load_image_file(path_img_to_recognize)
#     face_locations = face_recognition.face_locations(img_to_recognize, model="hog")
#     face_encodings = face_recognition.face_encodings(img_to_recognize, face_locations)
    
    img_to_recognize = load_image(path_img_to_identify)
    detected_faces = detector(img_to_recognize, 1) # определение области лица HOG
    
    if len(detected_faces) == 0:
        print("Error: There is no faces!")
        return
    
    # извлечение 128 вектора из фото для идентификации
    encoded_face = detect_face_and_encode(path_img_to_identify, "", image_identify=True)
    
    # KNN predict
    check = classifier.predict_proba([encoded_face])
    print(check)
    
    box_face = rect_to_box(detected_faces[0])
        
    name_of_person = "Людина не ідентифікована"

    # =========== Использование KNN ===========
    value_predict = np.amax(check)
    value_index = check.argmax()
    if value_predict > 0.6:
        flag_found = True
        name_of_person = get_unique_values(known_face_names)[value_index]
        
    # =========================================

    cv2.rectangle(img_to_recognize, (box_face[0], box_face[1]), (box_face[2], box_face[3]), (0, 255, 0), 2)
    font = cv2.FONT_HERSHEY_DUPLEX
    #cv2.putText(img_to_recognize, name_of_person, (box_face[0], box_face[3]), font, 0.5, (255, 255, 255), 1)
    
    # ============ ОТОБРАЖЕНИЕ В ОКНЕ ПРИЛОЖЕНИЯ ===============
    # отрисовка слева фото с выделенным лицом
    img_recognize = Image.fromarray(img_to_recognize)
    img_recognize.thumbnail((400,400))
    img_recognize = ImageTk.PhotoImage(img_recognize)
    photo_left.configure(image=img_recognize)
    photo_left.image = img_recognize
        
    # Если человек найден - отобразить его фото из БД
    if flag_found == True:
        img_in_db = Image.open("images_db/" + name_of_person + "/" + name_of_person + ".jpg")
        img_in_db.thumbnail((400,400))
        img_in_db = ImageTk.PhotoImage(img_in_db)
        photo_right.configure(image=img_in_db)
        photo_right.image = img_in_db
    else: # Иначе отобразить фото Инкогнито
        unknown_img = Image.open("Unknown_person.png")
        unknown_img.thumbnail((400,400))
        unknown_img = ImageTk.PhotoImage(unknown_img)
        photo_right.configure(image=unknown_img)
        photo_right.image = unknown_img
        
    text_name_person.config(text="Имя: " + name_of_person)

# Окно приложения

In [22]:
root = Tk()
root.title("Ідентифікація людини")
root.geometry("1000x600")
root.resizable(width=False, height=False)
root.configure(background='#c2d6d6')

# =========== LEFT SIDE ==============
frame_left = Frame(root, bg='gray')
frame_left.place(relx=0.01, rely=0.05, relwidth=0.47, relheight=0.8)

button_choose = Button(frame_left, text="Обрати зображення", command=show_image, background="#527a7a", font="Arial 14")
button_choose.place(relx=0.1, rely=0.85, relwidth=0.3, relheight=0.07)

photo_left = Label(frame_left)
photo_left.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.7)

text_id_photo = Label(frame_left, text="Зображення для ідентифікації", font="Arial 16", bg='gray', fg='white')
text_id_photo.pack(side=tk.TOP)

# =========== RIGHT SIDE ==============
frame_right = Frame(root, bg='gray')
frame_right.place(relx=0.51, rely=0.05, relwidth=0.48, relheight=0.8)

photo_right = Label(frame_right)
photo_right.place(relx=0.02, rely=0.1, relwidth=0.96, relheight=0.7)

text_right_photo = Label(frame_right, text="Знайдена людина у БД", font="Arial 16", bg='gray', fg='white')
text_right_photo.pack(side=tk.TOP)

text_name_person = Label(frame_right, text="Им'я: ", font="Arial 16", bg='gray', fg='white')
text_name_person.place(relx=0.05, rely=0.85)

button_add_person = Button(frame_right, text="Додати людину до БД", command=add_person_to_db, background="#527a7a", font="Arial 14")
button_add_person.place(relx=0.58, rely=0.92, relwidth=0.4, relheight=0.07)

# ========== TOP RESULTS ===========
# text_distance = Label(root, text="Расстояние: 0.0", bg='gray')
# text_distance.pack(side=tk.TOP)

button_recognize = Button(root, text="Ідентифікувати", command=recognize_person, background="#527a7a", font="Arial 16")
button_recognize.place(relx=0.35, rely=0.9, relwidth=0.3, relheight=0.08)

root.mainloop()